<a href="https://colab.research.google.com/github/stong1108/nyt-recipe-mods/blob/main/recipe_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install openai beautifulsoup4

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
from google.colab import userdata
from openai import OpenAI
client = OpenAI(api_key = userdata.get('openai_api_key'))

In [4]:
def get_completion(prompt, model='gpt-3.5-turbo'):
    messages = [{"role": "system", "content": "You are a cooking enthusiast."},{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content


In [5]:
def extract_content(recipe_url):
  r = requests.get(recipe_url)
  if r.status_code != 200:
    raise ValueError(f'Bad status code: {r.status_code}')

  soup = BeautifulSoup(r.content)
  section = soup.find('section', {'role': 'tabpanel', 'aria-labelledby': 'helpful'})
  n_children = len(list(section.children))
  if n_children < 5:
    raise ValueError('Not enough helpful comments posted')
  most_helpful_comments = '\n'.join([child.p.text for child in section.children])
  recipe_name = soup.title.text.replace(' - NYT Cooking', '')

  return recipe_name, most_helpful_comments

In [6]:
prompt = """Provided below is a list of the most helpful comments on a recipe
for {}. Summarize the top 5 recipe modifications in a bulleted list.
Only output the bulleted list and nothing else.

Most helpful comments:
{}
"""

In [11]:
def recipe_suggestions(recipe_url, prompt=prompt):
  try:
    recipe_name, most_helpful_comments = extract_content(recipe_url)
    suggestions = get_completion(prompt.format(recipe_name, most_helpful_comments))
    print(f'Suggested improvements for {recipe_name}')
    print(suggestions)
  except ValueError as e:
    print(f'Unable to generate due to error- {e}')

In [12]:
recipe_url = 'https://cooking.nytimes.com/recipes/1025054-skillet-ginger-chicken-with-apricots'

recipe_suggestions(recipe_url)

Unable to generate due to error- Not enough helpful comments posted


In [13]:
recipe_url = 'https://cooking.nytimes.com/recipes/1025042-microwave-salmon'

recipe_suggestions(recipe_url)

Suggested improvements for Microwave Salmon Recipe
- Use Italian dressing instead of water for added flavor
- Add garlic powder, lemon juice, bay leaf, and thyme for enhanced taste
- Squeeze lime on the fillet before cooking to reduce fish odor
- Use an insta-read thermometer to prevent overcooking
- Brush the bottom of the dish with olive oil and add white vermouth for extra flavor
